In [1]:
import pandas as pd 
import os
import numpy as np

- Census data extracted from four years (2000, 2011, 2016, 2021) and then linearly interpolated 
- Census data kept inside assets/

This notebook only interpolates 'population', listed by column name B01003_001E, but workflow should translate to interpolate across all columns. Will need to explore more efficient methods 

-  Workflow:
    - read four files
    - display population across years
    - create three separate dataframes that interpolate between (2000-2011, 2011-2016, 2016-2021)
    - merge and create a final dataframe that has populations across all zip codes from 2000-2021


In [2]:
census_files = ['../assets/census/'+i for i in os.listdir('../assets/census/')]
dec_2000 = pd.read_csv([i for i in census_files if '2000' in i][0])
acs_2011 = pd.read_csv([i for i in census_files if '2011' in i][0])
acs_2016 = pd.read_csv([i for i in census_files if '2016' in i][0])
acs_2021 = pd.read_csv([i for i in census_files if '2021' in i][0])

In [3]:
dec_2000

,PAT_ZIP,B01003_001E,B04007_001E,B04007_002E,B04007_003E,B04007_004E,B04007_005E,B04007_006E,B04007_007E,B10050_001E,...,B99255_003E,B99256_001E,B99256_002E,B99256_003E,B99257_001E,B99257_002E,B99257_003E,B99258_001E,B99258_002E,B99258_003E
0,73949,1866,1866,1273,937,336,593,16,577,1013,...,780,814,94,720,696,48,648,814,57,757
1,75001,8175,8175,6827,4819,2008,1348,89,1259,4974,...,5180,5299,396,4903,4944,275,4669,5299,377,4922
2,75002,37171,37171,29120,19626,9494,8051,446,7605,19867,...,12379,12673,654,12019,12262,540,11722,12673,761,11912
3,75006,45990,45990,38151,30811,7340,7839,388,7451,25298,...,16632,17546,2643,14903,16880,1100,15780,17546,2200,15346
4,75007,50357,50357,41933,32073,9860,8424,335,8089,28681,...,17222,17768,1250,16518,17421,847,16574,17768,1610,16158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1831,79936,92136,92136,84919,78409,6510,7217,184,7033,43197,...,27494,28238,2344,25894,26890,826,26064,28238,4190,24048
1832,79938,18379,18379,14846,14402,444,3533,12,3521,6675,...,4422,4486,489,3997,4062,110,3952,4486,846,3640
1833,88063,10274,10274,9239,9139,100,1035,38,997,4607,...,2654,2734,410,2324,2566,153,2413,2734,418,2316
1834,88220,33185,33185,26160,20873,5287,7025,304,6721,19008,...,14045,14511,1262,13249,12625,525,12100,14511,1238,13273


In [15]:
dec_2000.loc[:,[i for i in dec_2000.columns if 'B19013' in i]].describe()

,B19013_001E,B19013A_001E,B19013B_001E,B19013C_001E,B19013D_001E,B19013E_001E,B19013F_001E,B19013G_001E,B19013H_001E,B19013I_001E
count,1836.000000,1836.000000,1836.000000,1836.000000,1836.000000,1836.000000,1836.000000,1836.000000,1836.000000,1836.000000
mean,37813.182462,39367.096950,24261.949891,28454.734205,27655.327887,8045.627451,28977.188998,29318.714052,41185.168301,31036.900871
std,16021.288202,16358.330569,24368.140690,30066.324768,35291.888772,22526.538752,19427.709653,24428.717740,16898.858252,17469.866568
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,28281.000000,30048.000000,3750.000000,0.000000,0.000000,0.000000,19617.250000,14474.500000,31640.500000,21875.000000
50%,33997.000000,35884.000000,20008.500000,23819.500000,13798.000000,0.000000,27537.000000,27389.500000,37500.000000,28661.000000
75%,42935.500000,44613.750000,34272.000000,43341.000000,48750.000000,0.000000,36250.000000,39689.000000,46485.750000,37780.750000
max,200000.000000,200001.000000,200001.000000,200001.000000,200001.000000,200001.000000,200001.000000,200001.000000,200001.000000,200001.000000


In [20]:
variable = 'B19013_001E'

In [22]:
int_df = pd.merge(
            pd.merge(
                pd.merge(
                    dec_2000.loc[:,['PAT_ZIP',variable]],acs_2011.loc[:,['PAT_ZIP',variable]],'outer',on='PAT_ZIP', suffixes=['_2000','_2011'])
                ,acs_2016.loc[:,['PAT_ZIP',variable]], 'outer',on='PAT_ZIP',suffixes=['','_2016'])
            ,acs_2021.loc[:,['PAT_ZIP',variable]], 'outer',on='PAT_ZIP',suffixes=['','_2021'])
int_df.columns = ['PAT_ZIP','2000','2011','2016','2021']
int_df

,PAT_ZIP,2000,2011,2016,2021
0,73949,33150.0,NaN,NaN,53491.0
1,75001,57064.0,55874.0,70020.0,73383.0
2,75002,74144.0,91638.0,96060.0,110796.0
3,75006,52798.0,57274.0,56577.0,73088.0
4,75007,70081.0,78660.0,78488.0,96051.0
...,...,...,...,...,...
1934,79911,NaN,140710.0,83889.0,104145.0
1935,79916,NaN,24211.0,-666666666.0,-666666666.0
1936,79920,NaN,-666666666.0,-666666666.0,-666666666.0
1937,79928,NaN,44886.0,49953.0,62471.0


In [30]:
int_df[int_df > 0].describe()

,PAT_ZIP,2000,2011,2016,2021
count,1939.00000,1832.000000,1874.000000,1797.000000,1750.000000
mean,77406.01444,37895.743996,49630.073639,53722.236505,65085.104000
std,1473.99288,15940.890270,21397.422078,22427.429926,26429.877081
min,73949.00000,4444.000000,2499.000000,13105.000000,2499.000000
25%,76131.50000,28299.000000,36541.750000,39705.000000,47743.000000
50%,77436.00000,34011.000000,45133.500000,48725.000000,60313.000000
75%,78605.50000,42979.250000,58121.500000,61483.000000,76754.000000
max,88430.00000,200000.000000,238208.000000,250001.000000,240694.000000


### Helper functions to extract population data, interpolate, clean etc.
- Use CleanAndMerge for 2000 to 2011, use CleanAndMergeMiddle for the rest

In [34]:
def cleanAndMerge(df1, df2, year1, year2, variable):
    df1 = df1.loc[:,['PAT_ZIP',variable]]
    df2 = df2.loc[:,['PAT_ZIP',variable]]
    df = pd.merge(df1, df2, 'outer',on='PAT_ZIP')
    df.columns = ['PAT_ZIP', str(year1), str(year2)]

    for i,j in enumerate(range(year1+1, year2)):
        df.insert(i+2, j, np.NaN)
    
    fin_df = df.iloc[:,1:].interpolate(axis=1, limit_direction='both').round().astype(int)
    fin_df.insert(0, 'PAT_ZIP', df['PAT_ZIP'])

    return fin_df

def cleanAndMergeMiddle(df1, df2, year1, year2, variable):
    df1 = df1.loc[:,['PAT_ZIP',str(year1)]]
    df2 = df2.loc[:,['PAT_ZIP',variable]]

    df = pd.merge(df1, df2, 'outer',on='PAT_ZIP')
    df.columns = ['PAT_ZIP', str(year1), str(year2)]

    for i,j in enumerate(range(int(year1+1), year2)):
        df.insert(df1.shape[1]+i, j, np.NaN)
    
    fin_df = df.iloc[:,1:].interpolate(axis=1, limit_direction='both').round().astype(int)
    fin_df.insert(0, 'PAT_ZIP', df['PAT_ZIP'])

    return fin_df

In [35]:
def interpolateVariable(var):
    merge_0011 = cleanAndMerge(dec_2000, acs_2011, 2000, 2011, var)
    merge_1116 = cleanAndMergeMiddle(merge_0011, acs_2016, 2011, 2016, var)
    merge_1621 = cleanAndMergeMiddle(merge_1116, acs_2021, 2016, 2021, var)
    interpolatedPopulations = pd.merge(
                                pd.merge(
                                    merge_0011.drop('2011',axis=1), merge_1116, on='PAT_ZIP').drop('2016',axis=1), 
                                merge_1621, on='PAT_ZIP')
    interpolatedPopulations.to_csv(f'../assets/census/{var}.csv', index=False)



In [36]:
interpolateVariable(variable)

In [6]:
merge_0011 = cleanAndMerge(dec_2000, acs_2011, 2000, 2011)
merge_1116 = cleanAndMergeMiddle(merge_0011, acs_2016, 2011, 2016)
merge_1621 = cleanAndMergeMiddle(merge_1116, acs_2021, 2016, 2021)

In [7]:
interpolatedPopulations = pd.merge(
                            pd.merge(
                                merge_0011.drop('2011',axis=1), merge_1116, on='PAT_ZIP').drop('2016',axis=1), 
                                merge_1621, on='PAT_ZIP')
interpolatedPopulations

,PAT_ZIP,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,73949,1866,1866,1866,1866,1866,1866,1866,1866,1866,...,1866,1866,1866,1866,1866,1764,1662,1561,1459,1357
1,75001,8175,8569,8963,9357,9751,10145,10540,10934,11328,...,12892,13274,13657,14039,14421,14794,15167,15541,15914,16287
2,75002,37171,39444,41717,43990,46263,48536,50810,53083,55356,...,63513,64850,66188,67525,68863,69311,69759,70206,70654,71102
3,75006,45990,45985,45979,45974,45969,45963,45958,45952,45947,...,46752,47574,48395,49217,50038,49651,49264,48878,48491,48104
4,75007,50357,50408,50460,50511,50562,50613,50665,50716,50767,...,51599,52277,52956,53634,54312,54339,54366,54394,54421,54448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1934,79911,1849,1849,1849,1849,1849,1849,1849,1849,1849,...,2261,2673,3084,3496,3908,4660,5412,6164,6916,7668
1935,79916,3144,3144,3144,3144,3144,3144,3144,3144,3144,...,3109,3074,3038,3003,2968,3280,3591,3903,4214,4526
1936,79920,52,52,52,52,52,52,52,52,52,...,51,50,48,47,46,38,30,23,15,7
1937,79928,44205,44205,44205,44205,44205,44205,44205,44205,44205,...,46847,49489,52131,54773,57415,61210,65006,68801,72597,76392


In [8]:
interpolatedPopulations.to_csv('../assets/census/interpolatedPopulations.csv', index=False)